Import packages.

In [9]:
import os
import sys

import torch

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from signal_autoencoder.dataloading import collate_fn
from signal_autoencoder.modeling import (ConvAutoencoder,
                                         ConvRecAutoencoder,
                                         count_parameters,
                                         sequence_l1)

Let's create test data: a batch of three 8-channel signals of different length.

In [10]:
n_channel = 8

x1 = torch.rand((1000, n_channel))  # Length: 1000
x2 = torch.rand((2000, n_channel))  # Length: 2000
x3 = torch.rand((2999, n_channel))  # Length: 2999

Then gather the signals, pad them to the batch max length, store the original signal lengths, create a mask to recover later which elements are padded.

In [11]:
x_in, lengths, mask = collate_fn([(x1, 1000), (x2, 2000), (x3, 3000)])
print('Input tensor size:', x_in.size())

Input tensor size: torch.Size([3, 2999, 8])


Define the convolutive autoencoder model.

In [12]:
n_conv_channel_1 = 128
n_conv_channel_2 = 256
n_conv_channel_3 = 512
n_conv_channel_4 = 1024

autoenc = ConvAutoencoder(n_channel, n_conv_channel_1, n_conv_channel_2,
                          n_conv_channel_3, n_conv_channel_4)

print(f'The model has {count_parameters(autoenc)} parameters')

The model has 9362160 parameters


Define the convolutive recurrent autoencoder model.

In [13]:
n_conv_channel_1 = 64
n_conv_channel_2 = 128
n_conv_channel_3 = 256
lstm_hidden_size = 512
n_lstm_layer = 2

autoenc = ConvRecAutoencoder(n_channel, n_conv_channel_1,
                             n_conv_channel_2, n_conv_channel_3,
                             lstm_hidden_size, n_lstm_layer)

print(f'The model has {count_parameters(autoenc)} parameters')

The model has 8584432 parameters


In [14]:
x_out = autoenc(x_in)

print('Output tensor size:', x_out.size())

Output tensor size: torch.Size([3, 2992, 8])


Compute the L1 distance between the original input and the reconstructed output. Earlier variables `lengths` and `mask` are used here, to discard padded elements from the loss computation.

In [15]:
err = sequence_l1(x_in, x_out, lengths, mask)

print('L1 distance:', err.item())

L1 distance: 5.819143772125244


This distance can be used as a loss for training the autoencoder.

Next, get the codes (compressed representations) corresponding to the batch input.

In [16]:
with torch.no_grad():
    _, (code, _) = autoenc.encoder(x_in)
code = code.permute((1, 0, 2))  # Batch dimension first
code = code.flatten(start_dim=1)
print('Code size:', code.size())

Code size: torch.Size([3, 1024])


In the convolutional recurrent autoencoder, the latent space dimension, i.e. the code size, is `lstm_hidden_size * n_lstm_layer`, here `512 * 2 = 1024`.